In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

image = cv2.imread('/mnt/f/valorant_VODs/gray_roi_4.png', cv2.IMREAD_GRAYSCALE)


kernel = np.ones((5,5),np.uint8)


thresholds = list(range(80, 110))


plt.figure(figsize=(5, 5))
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.show()


n = len(thresholds)
cols = 5
rows = n // cols + 1

fig, ax = plt.subplots(rows, cols, figsize=(20, rows*4))

for i, threshold in enumerate(thresholds):
    binary_thresh = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)[1]
    dilated = cv2.dilate(binary_thresh, kernel, iterations = 1)

    mask = binary_thresh > 0
    image_copy = image.copy()
    image_copy[~mask] = 0

    row = i // cols
    col = i % cols
    ax[row, col].imshow(image_copy, cmap='gray')
    ax[row, col].set_title('Threshold = ' + str(threshold))

for j in range(i+1, rows*cols):
    fig.delaxes(ax.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import pytesseract
import numpy as np
from matplotlib import pyplot as plt
from collections import deque, Counter

ocr_roi = (978, 7, 1000, 24)
crop_roi = (30, 50, 410, 410)


cap = cv2.VideoCapture("/mnt/f/valorant_VODs/test2.mp4")

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

fps = 3
width = crop_roi[2] - crop_roi[0]
height = crop_roi[3] - crop_roi[1]

video_writers = {}
current_number = 1
frame_counter = 0

def preprocess_image(image):
    threshold = 103
    binary_thresh = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)[1]
    mask = binary_thresh > 0
    image_copy = image.copy()
    image_copy[~mask] = 0
    return image_copy

def numbercheck(number):
    number = number.strip()
    if number.isdigit():
        num = int(number)
        return 0 <= num - int(current_number) <= 1
    return False


buffer_size = 10
buffer = deque(maxlen=buffer_size)

while(cap.isOpened()):
    ret, frame = cap.read()
    frame_counter += 1
    if ret == True:
        if frame_counter % 10 == 0:
            
            roi = frame[ocr_roi[1]:ocr_roi[3], ocr_roi[0]:ocr_roi[2]]
            gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray_roi_processed = preprocess_image(gray_roi)
            number = pytesseract.image_to_string(gray_roi_processed, config='--psm 6 -c tessedit_char_whitelist=0123456789')
            
            # print(number,end=' ')
            # if frame_counter % 50 == 0:
            #     print(frame_counter)

            if numbercheck(number):
                buffer.append(number.strip())
            
            
            if buffer:
                most_common_number, count = Counter(buffer).most_common(1)[0]
                
                if count/buffer_size >= 0.5:
                    current_number = most_common_number
                
                # writing the frame to the video
                if current_number not in video_writers:
                    # print('number:'+current_number)
                    video_writers[current_number] = cv2.VideoWriter('/mnt/f/valorant_VODs/test2_round' + str(current_number).strip() + '.mp4', fourcc, fps, (width, height))

                cropped_frame = frame[crop_roi[1]:crop_roi[3], crop_roi[0]:crop_roi[2]]
                video_writers[current_number].write(cropped_frame)
    else:
        break

cap.release()
for writer in video_writers.values():
    writer.release()

cv2.destroyAllWindows()